In [1]:
import pandas as pd             # data package
import matplotlib.pyplot as plt # graphics 
import datetime as dt
import numpy as np
from census import Census # This is new...

import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import os  

#import weightedcalcs as wc
#import numpy as np

import pyarrow as pa
import pyarrow.parquet as pq

from bokeh.palettes import brewer, Spectral6
from bokeh.io import show, output_file, curdoc
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool, Panel, Tabs, GeoJSONDataSource, LinearColorMapper
from bokeh.models import ColorBar
from bokeh.layouts import column, gridplot, row
from bokeh.transform import factor_cmap
from bokeh.models import NumeralTickFormatter, Title, Label, Paragraph, Div, CustomJSHover, BoxAnnotation


In [2]:
country_list = pd.read_csv('./data/top_50_non_eu.csv', dtype={'CTY_CODE': str})

country_list.loc[len(country_list)] = ["EUROPEAN UNION", 0 , "0003"]

country_list["tariff"] = 10.0

country_list.loc[country_list["CTY_CODE"] == "5700", "tariff"] = 145.0

country_list.loc[country_list["CTY_CODE"] == "1220", "tariff"] = 8.75

country_list.loc[country_list["CTY_CODE"] == "2010", "tariff"] = 4.5


In [3]:
country_list

,CTY_NAME,total_imports,CTY_CODE,tariff
0,CHINA,5.728638e+12,5700,145.00
1,MEXICO,4.385957e+12,2010,4.50
2,CANADA,4.161438e+12,1220,8.75
3,JAPAN,1.701399e+12,5880,10.00
4,"KOREA, SOUTH",1.045098e+12,5800,10.00
5,VIETNAM,8.800542e+11,5520,10.00
6,TAIWAN,7.522322e+11,5830,10.00
7,INDIA,7.405954e+11,5330,10.00
8,UNITED KINGDOM,7.096412e+11,4120,10.00
9,SWITZERLAND,6.056859e+11,4419,10.00


Step one is to read in the tariff lists, these come from the orginal Annex's published in the Federal Register

In [ ]:
expemption_list = pd.read_csv('./tariff-lists/annex-II-exemptions.csv', dtype={'HTSUS': str})
# these are the annex II exemptions from the reciprical tariff list

expemption_list_phones = pd.read_csv('./tariff-lists/41225_exemptions.csv', dtype={'HTSUS': str})
# these are the phone exemptions

steel_list = pd.read_csv('./tariff-lists/steel-tariffs.csv', dtype={'HTSUS': str})
# these are the steel products subject to the new 232 tariffs. This is at 8 digit level.

alu_list = pd.read_csv('./tariff-lists/alu-tariffs.csv', dtype={'HTSUS': str})
# these are the aluminum products subject to the new 232 tariffs. This is at a mixed 8 and 10 digit level.

alu_8_list = alu_list[alu_list['HTSUS'].str.len() == 8]

auto_list = pd.read_csv('./tariff-lists/auto-tariffs.csv', dtype={'HTSUS': str})
# these are the aluminum products subject to the new 232 tariffs. This is at a mixed 8 and 10 digit level.

auto_6_list = auto_list[auto_list['HTSUS'].str.len() == 6]

auto_8_list = auto_list[auto_list['HTSUS'].str.len() == 8]


# country_list = pd.read_csv('./tariff-lists/reciprocal-tariffs-annex-I.csv',dtype={'CTY_CODE': str})

In [ ]:
def get_all_imports(cnty_code, tariff):

    my_key = "&key=34e40301bda77077e24c859c6c6c0b721ad73fc7"

    hs = 'HS10'

    my_key = "&key=34e40301bda77077e24c859c6c6c0b721ad73fc7"

    end_use = "hs?get=CTY_NAME,CON_VAL_YR,I_COMMODITY,I_COMMODITY_SDESC"

    surl = "https://api.census.gov/data/timeseries/intltrade/imports/" + end_use 

    surl  = surl + my_key + "&time=" + "2024-12" + "&COMM_LVL=" + hs 

    url = surl + "&CTY_CODE=" + cnty_code

    r = requests.get(url) 
    
    print(r)
    
    df = pd.DataFrame(r.json()[1:]) # This then converts it to a dataframe
    # Note that the first entry is the labels

    df.columns = r.json()[0]

    #######################################################################

    df.time = pd.to_datetime(df.time, format="%Y-%m")

    df["imports"] = df["CON_VAL_YR"].astype(float)

    df["share"] = df["imports"] / df["imports"].sum()
            
    df[hs] = df["I_COMMODITY"].astype(str)

    df["HS8"] = df["HS10"].str[0:8]

    df["HS6"] = df["HS10"].str[0:6]

    #######################################################################
    # this then filters so products are only those that are subject to the new tariffs
    # the reciprocal exemptions, steel, aluminum and auto tariffs.

    df["tariff"] = tariff

    # if cnty_code == "5700": # if china then big tariff

    #     df["tariff"] = 125.0

    # elif cnty_code == "1220" or cnty_code == "2010": 

    #     df["tariff"] = tariff


    exemption = df["HS10"].isin(expemption_list["HTSUS"].tolist()) | df["HS10"].isin(expemption_list_phones["HTSUS"].tolist())

    df.loc[exemption, "tariff"]  = 0.0

    exempt_imports = df.loc[exemption, "imports"].sum()

    ####################################################


    steel = df["HS8"].isin(steel_list["HTSUS"].tolist())

    df.loc[steel, "tariff"]  = 25.0

    steel_imports = df.loc[steel, "imports"].sum()

    ####################################################

    alu = df["HS10"].isin(alu_list["HTSUS"].tolist()) | df["HS8"].isin(alu_8_list["HTSUS"].tolist())
       
    df.loc[alu, "tariff"]  = 10.0

    alu_imports = df.loc[alu, "imports"].sum()

    ####################################################


    auto = df["HS8"].isin(auto_8_list["HTSUS"].tolist()) | df["HS10"].isin(auto_list["HTSUS"].tolist()) | df["HS6"].isin(auto_6_list["HTSUS"].tolist())

    if cnty_code != "1220" and cnty_code != "2010": # if canada or mexico, then assumption is no tariff

        df.loc[auto, "tariff"]  = 25.0

    auto_imports = df.loc[auto, "imports"].sum()

    ####################################################


    return df, exempt_imports, steel_imports, alu_imports, auto_imports

In [6]:
country_list

,CTY_NAME,total_imports,CTY_CODE,tariff
0,CHINA,5.728638e+12,5700,145.00
1,MEXICO,4.385957e+12,2010,4.50
2,CANADA,4.161438e+12,1220,8.75
3,JAPAN,1.701399e+12,5880,10.00
4,"KOREA, SOUTH",1.045098e+12,5800,10.00
5,VIETNAM,8.800542e+11,5520,10.00
6,TAIWAN,7.522322e+11,5830,10.00
7,INDIA,7.405954e+11,5330,10.00
8,UNITED KINGDOM,7.096412e+11,4120,10.00
9,SWITZERLAND,6.056859e+11,4419,10.00


In [10]:
df = pd.DataFrame()

recipricol_summary = pd.DataFrame(columns=['country_name','total imports',"effective tariff", "expempt imports", "steel imports", "alu imports", "auto imports"])

for index, row in country_list.iterrows():
    

    foo, exempt_imports, steel_imports, alu_import, auto_imports = get_all_imports(row['CTY_CODE'], row['tariff'])

    recipricol_summary.loc[index, 'country_name'] = foo['CTY_NAME'].iloc[0]
    
    recipricol_summary.loc[index, 'total imports'] = foo['imports'].sum()

    recipricol_summary.loc[index, 'effective tariff'] = ( foo['tariff'] * foo['share'] ).sum()

    recipricol_summary.loc[index, 'expempt imports'] = exempt_imports
    
    recipricol_summary.loc[index, 'steel imports'] = steel_imports
    
    recipricol_summary.loc[index, 'alu imports'] = alu_import
    
    recipricol_summary.loc[index, 'auto imports'] = auto_imports    


    df = df.append(foo)

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


In [11]:
recipricol_summary.sort_values(by='total imports', ascending=False, inplace=True)

In [12]:
avg_tariff = ( recipricol_summary["effective tariff"]*recipricol_summary["total imports"] ).sum() / recipricol_summary["total imports"].sum() + 2.3 

print("Average Tariff Rate: ", avg_tariff)

Average Tariff Rate:  27.719374390059766


In [13]:
def make_empty_df():

    empty_df = pd.DataFrame(columns=['country_name','total imports',"effective tariff", "expempt imports", "steel imports", "alu imports", "auto imports"])

    empty_df.loc[0, 'country_name'] = ''
    
    empty_df.loc[0, 'total imports'] = 0.0

    empty_df.loc[0, 'effective tariff'] = 0.0

    empty_df.loc[0, 'expempt imports'] = 0.0

    empty_df.loc[0, 'steel imports'] = 0.0

    empty_df.loc[0, 'alu imports'] = 0.0

    empty_df.loc[0, 'auto imports'] = 0.0

    return empty_df

In [14]:
def make_source(df):
    
    df["position"] = df.reset_index().index.values
        
    df["hover_label"] = (df["total imports"]/1000000000).map('{:,.1f}'.format)
        
    df["hover_label_2"] = (df["effective tariff"]).map('{:,.1f}'.format)
    
    source = ColumnDataSource(df)
    
    return source

In [15]:
def make_bar_chart(df):

    height = int(1.15*533)
    width = int(1.15*750)

    source = make_source(df)
        
    p = figure(plot_height=height, plot_width = width, title= "U.S. Tariffs by Country of Origin (Top 20 by Import Value)", x_range=df['country_name'],
           toolbar_location = 'below',
           tools = "reset")
            
    p.vbar(x = "country_name", top = "effective tariff", width = 0.6, alpha = 0.65,
       hatch_pattern = " ",hatch_alpha = 0.10, color = "red",
       source = source)
    
    y_custom = CustomJSHover(code=""" return '' + special_vars.data_y
            """)

##########################################################################
    TIMETOOLTIPS = """
    <div style="background-color:#F5F5F5; opacity: 0.95; border: 0px 0px 0px 0px">
        <div style = "text-align:left;">
            <span style="font-size: 13px; font-weight: bold">@country_name</span>
        </div>
        <div style = "text-align:left;">
            <span style="font-size: 13px; font-weight: bold">2024 Imports: $@hover_label Billion</span>
        </div>
        <div style = "text-align:left;">
            <span style="font-size: 13px; font-weight: bold">Applied Tariff: @hover_label_2%</span>
        </div>
    </div>
    """

    p.add_tools(HoverTool(tooltips = TIMETOOLTIPS))
##########################################################################

    mytext = Label(x=400, y=410, text='''U.S. Average Tariff:''', text_font_size="2em", text_font_style = "bold",
                      x_units='screen', y_units='screen')

    p.add_layout(mytext)
    mytext = Label(x=400, y=320, text= str(round(avg_tariff,1))+ '%', text_font_size="6em", text_font_style = "bold",
                      x_units='screen', y_units='screen')
    p.add_layout(mytext)

    #p.ygrid.grid_line_color = None
    p.xgrid.grid_line_color = None
    
    p.title.text_font_size = '14pt'
    p.xaxis.major_tick_line_color = None  # turn off x-axis major ticks
    p.xaxis.minor_tick_line_color = None  # turn off x-axis minor ticks


    p.xaxis.major_label_text_font_size = '7pt'  # turn off x-axis tick labels
    p.xaxis.major_label_orientation = 0.75 

    p.yaxis.formatter = NumeralTickFormatter(format="(0. a)")
    p.yaxis.minor_tick_line_color = None
    p.y_range.start = 0 
    
    p.y_range.end = df["effective tariff"].max() + 0.10*df["effective tariff"].max()
    
    p.border_fill_color = background    
    
    p.background_fill_color = background 
    p.background_fill_alpha = 0.75    
    
    p.toolbar.autohide = True
    
    p.outline_line_color = None
    p.sizing_mode= "scale_both"
    p.max_height = height
    p.max_width = width
    p.min_height = int(0.25*height)
    p.min_width = int(0.25*width)

    
    return p

In [16]:
crl = ["darkblue","slategray","slategray","crimson","crimson"]

background = "#ffffff"

In [17]:
foo = recipricol_summary[0:20].copy(deep=True)

foobar = make_empty_df()

foobar = foobar.append(foo)

p1 = make_bar_chart(foobar)

tab1 = Panel(child= p1, title="")

output_file('.\\docs\\' + "reciprocal-china.html")

div0 = Div(text = """Each bar represents the trade-weighted applied tariff for each country. Applied tariffs include reciprocal tariffs (10% all except China, Canada, and Mexico), 
           steel and aluminum tariffs, and auto tariffs. Fentanyl tariffs for Canada and Mexico are calculated under the assumption that all auto 
           imports are USMCA-compliant, and that 65% of Canadian imports and 82% of Mexican imports are USMCA-compliant. The average tariff 
           reflects the trade-weighted average of all tariffs, including the existing 2.3% applied tariff as of February 2025.
""", max_width= int(1.15*750), background = background )

div0.sizing_mode= "scale_both"
        
outfig = column(p1, div0, sizing_mode="scale_both")

show(outfig)

In [18]:
world, exempt_imports, steel_imports, alu_import, auto_imports = get_all_imports('-', 10.0)

<Response [200]>


In [19]:
recipricol_summary["total imports"].sum() / world.imports.sum() 

0.9894929660236773

In [ ]:
( world.imports.sum() - recipricol_summary["total imports"].sum() ) / world.imports.sum()

0.10480253116967053

In [ ]:
( steel_imports - recipricol_summary["steel imports"].sum() ) / steel_imports

0.052666772330590944

In [ ]:
(alu_import - recipricol_summary["alu imports"].sum() ) / alu_import

0.0567159109554093

In [ ]:
(auto_imports - recipricol_summary["auto imports"].sum() ) / auto_imports

0.038045350632093015